In [0]:
pip install -U openai

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-8d65db29-6d9c-43fd-b82c-72b63471243f
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
  Attempting uninstall: openai
    Found existing installation: openai 0.28.1
    Not uninstalling openai at /databricks/python3

In [0]:
dbutils.library.restartPython()

In [0]:
from openai import OpenAI

In [0]:
#### PERFORM TEXT TRANSLATION
DATABRICKS_TOKEN = DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
def perform_sentiment_analysis(review):
  # How to get your Databricks token: https://docs.databricks.com/en/dev-tools/auth/pat.html
  # DATABRICKS_TOKEN = os.environ.get('DATABRICKS_TOKEN')
  # Alternatively in a Databricks notebook you can use this:
  client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="<link-to-the-databricks-model-serving-endpoint>"
  )

  chat_completion = client.chat.completions.create(
    messages=[
    {
      "role": "system",
      "content": "I sell cameras all around the world. My customers have left a feedback. Rate each feedback from 1 to 5- 1 being very negative and 5 being very positive. Also, mention if the review is positive (represented by 1) or negative (represented by -1) or neutral (represented by 0). Also, extract the key highlights of the review within 25 words as a summary. Make sure it contains the key complalins or praises from the review. Your output should be of the format rating|sentiment|summary. Example: 4|1|This is a great camera. Absolutely do not saying anything else. Here is the review:"
    },
    {
      "role": "user",
      "content": f"{review}. Remember, don't use any other filler words. Your response must strictly be of the format rating|sentiment|summary."
    }
    ],
    model="databricks-meta-llama-3-1-70b-instruct",
    max_tokens=200
  )
  print("Sending request to Llama 3.1")
  return chat_completion.choices[0].message.content

In [0]:
from pyspark.sql.functions import udf, split, col
from pyspark.sql.types import StringType, IntegerType

# Register the translate_review function as a UDF
sentiment_analysis_udf = udf(perform_sentiment_analysis, StringType())

df_translated = spark.read.table("harshit_rai_genai_demo.products.camera_all_reviews_translated")

df_translated = df_translated.withColumn("raw_sentiment", sentiment_analysis_udf(df_translated.translated_review))
df_translated = df_translated.withColumn("rating", split(df_translated.raw_sentiment, "\|")[0]).withColumn("sentiment", split(df_translated.raw_sentiment, "\|")[1]).withColumn("summary", split(df_translated.raw_sentiment, "\|")[2])
df_translated = df_translated.withColumn("rating", col("rating").cast(IntegerType())).withColumn("sentiment", col("sentiment").cast(IntegerType()))

In [0]:
df_translated.write.mode("overwrite").saveAsTable("harshit_rai_genai_demo.products.camera_reviews_with_sentiment_analysis")